# Digit Recognizer - using RandomForest in R

### Các thành viên:
- Vũ Đặng Quỳnh Giang
- Nguyễn Hiền Nhung
- Nguyễn Tiến Duy
- Phan Tấn Thịnh

## 1. Giới thiệu
Notebook này sử dụng thuật toán RandomForest để dự toán ký tự số dựa trên hình ảnh.
RandomForest là một thuật toán thuộc nhóm Supervised Learning (Học có giám sát) dùng để phân chia dữ liệu (Classification) thành các nhóm riêng biệt.

In [ ]:
# DIGIT RECOGNIZER WITH PCA & RandomForest  

library(readr)

library(ggplot2)

library("randomForest")



#Đọc file

train = read.csv("/kaggle/input/digit-recognizer/train.csv")
test= read.csv("/kaggle/input/digit-recognizer/test.csv")



Xem sơ lược số dòng và cột của tập dataset 
* Với tập train: 
Ta thấy tập train có 42000 dòng, tương ứng với 42000 bức ảnh, và 785 cột, tương ứng với số pixel trong một bức ảnh.
* Với tập test:
Có 28000 dòng tương ứng với 28000 bức ảnh.

In [ ]:
dim(train)
dim(test)

Xem trong tập train có bao nhiêu bức ảnh được dán nhãn là ký tự 0, 1, 2, ... 9

In [ ]:
table(as.factor(train$label))

Vẽ plot để trực quan hóa số lượng hình ảnh được dán nhãn theo từng ký tự.

In [ ]:

ggplot(train,aes(x=as.factor(label),fill=label))+
  geom_bar(stat="count",color="white")+
  scale_fill_gradient(low="lightblue",high="pink",guide=FALSE)+
  labs(title="Digits in Train Data",x="Digits")


Ta biết rằng các nhãn dán là các ký tự được viết tay, vậy chúng nhìn như thế nào? Sau đây chúng em chọn ngẫu nhiên 50 dòng từ tập dữ liệu train và biến nó thành hình ảnh.

In [ ]:
sample <- sample(1:nrow(train),50)
var <- t(train[sample,-1])
var_matrix <- lapply(1:50,function(x) matrix(var[,x],ncol=28))
opar <- par(no.readonly = T)
par(mfrow=c(5,10),mar=c(.1,.1,.1,.1))

for(i in 1:50) {
  for(j in 1:28) {
    var_matrix[[i]][j,] <- rev(var_matrix[[i]][j,])
  }
  image(var_matrix[[i]],col=grey.colors(225),axes=F)
}

## Data Preprocessing
Ta thấy nhiều predictor có giá trị bằng 0, rất hiếm khi có giá trị khác nhau. Suy ra variance của nó gần như bằng 0. Điều này gây khó khăn cho việc dữ đoán. Vậy nên ta phải lược bỏ những predictor đó.

Trước tiên lại chia tập data train ra thành tập train nhỏ và tập test nhỏ

In [ ]:
#Tách dữ liệu thành 2 tập . Lấy ngẫu nhiên 80% dữ liệu để train

set.seed(1)

divtrain = sample(nrow(train), nrow(train)*0.8)

train = train[divtrain,]  # training dataset

valid = train[-divtrain,] # validation dataset

labeltr = as.factor(train[,1])

labelvl = as.factor(valid[,1])


# Xóa cột label

train = train[,-1]

valid = valid[,-1]


# chia tất cả các tập dữ liệu cho 255 để chuẩn hóa giá trị pixel, nằm trong phạm vi từ 0 đến 255 

train = train/255

valid= valid/255

test = test/255




Áp dụng PCA vào ma trận covariance để để giảm kích thước và xác định số component bao nhiêu là đủ cho model.




In [ ]:
# áp dụng PCA 


train_pca <- prcomp(train, retx=TRUE, center=TRUE)
summary(train_pca)

var <- train_pca$sdev^2/sum(train_pca$sdev^2)

varcum <- cumsum(var)

result <- data.frame(num=1:length(train_pca$sdev),

                     ex=var,

                     cum=varcum)




In [ ]:
#tiến hành đánh giá tất cả các tập dữ liệu với 30 thành phần được lựa chọn trên PCA

train_pt <- as.matrix(train) %*% train_pca$rotation[,1:30]

train <- cbind(labeltr,as.data.frame(train_pt))

test <- as.matrix(test) %*% train_pca$rotation[,1:30]

test <- as.data.frame(test)

valid <- as.matrix(valid) %*% train_pca$rotation[,1:30]

valid <- as.data.frame(valid)

## Train and Prediction
Sau đó ta có thể train model. Chúng em dựa vào thuật toán RandomForest để xây dựng classification model.

In [ ]:
# train model : Random Forest 

model = randomForest(labeltr ~., data= train, ntree=1000)




In [ ]:
# Đưa ra dự đoán model trên tập val

pred <- predict(model,valid,type="response")

prediction <- data.frame(ImageId=1:nrow(valid),Label=pred)
prediction= cbind(prediction,labelvl)
prediction



#  confussion matrix

mc = table(pred, labelvl)

mc



In [ ]:

# Dự đoán trên tập test và save
pred_test <- predict(model,test)

df_y_test = as.data.frame(pred_test)
df_y_test = data.frame(ImageId = seq(1,length(df_y_test$pred_test)), Label = df_y_test$pred_test)
df_y_test
write.csv(df_y_test, file = "/kaggle/working/file.csv", row.names=F)




